In [3]:
from pulp import *
from itertools import compress #for boolean masking
import numpy as np
import pandas as pd
import re, timeit #might not need regular expressions anymore
import matplotlib.pyplot as plt

quarter = "2QFY16"
name = "Reading Optimization for %s" % quarter
hrs_wk = 3      #reading hours avail per week
wks = 13        #weeks per period (quarter)
read_speed = 80 #pages per hour

#hardcoding magic numbers
#mins and max numbers of book, by type
max_math = float('inf')
min_math = 2

max_hist = 2
min_hist = 1

max_phil = 1
min_phil = 1

#currently a minimum req'd percentage
percent_scifi = .3
#currently max percent already read
percent_read = .1

##max and min scifi
# <-----------------------

#eval versus production versions (solve repeatedly, or once)
#insert user interface- message box, auto-scrape of csv?
#set path to csv file? file_name?
#define solver dict to loop over cplex/gurobi/cbc/gplk
#need recency constraint


data = pd.read_csv('goodreads_library_export.csv')
p = pulp.LpProblem(name, pulp.LpMaximize)

data.shape

#binary decision variables for knapsack problem
dv = []

#define my sets/technological coefficients - Ratings
ratings = data["Average Rating"]
ratings = ratings.to_dict()
my_ratings = data["My Rating"]
#If I've rated a book, use my rating, not the average
for i in xrange(0,len(my_ratings)):
    if my_ratings[i] != 0:
        ratings[i] = my_ratings[i]

#Number of pages
pages = data["Number of Pages"]
pages = pages.to_dict()

#
bookshelves = data['Bookshelves']
math_books = bookshelves.isin(['math'])
milhist_books = bookshelves.isin(['military-history'])
phil_books = bookshelves.isin(['philosophy'])
read_books = data["Exclusive Shelf"].isin(['read'])

#init
total_pages= ""
total_books = ""
total_math = ""
total_phil = ""
total_milhist =""
per_scifi = ""
per_read = ""

#Sets and DVs
for rownum, row in data.iterrows():
    newDV = str(rownum)
    #relaxing the integer constraint could mean "read part/chapters of a book"
    newDV = pulp.LpVariable("x" + str(newDV), lowBound = 0, upBound = 1, cat= 'Integer')
    dv.append(newDV)
    total_pages   += dv[rownum] * pages[rownum]
    total_books   += (ratings[rownum] * int(pages[rownum])) * dv[rownum] 
   
    total_math    += int(math_books[rownum])    * dv[rownum]
    total_phil    += int(phil_books[rownum])    * dv[rownum]
    total_milhist += int(milhist_books[rownum]) * dv[rownum]
    
    per_scifi += (int(math_books[rownum])-(1-percent_scifi)) * dv[rownum]
    per_read +=  (int(read_books[rownum])-(1-percent_read) ) * dv[rownum]
    
#Objective Function
#defined as a utility function defined in units of "star-pages"
p += total_books
 


#constraints    
#Limit page count to available reading time
read_time_avail = hrs_wk * wks * read_speed
p += (total_pages <= read_time_avail)

#Hard limits on book numbers
p += (max_math >= total_math >= min_math)
p += (max_hist >= total_milhist >= min_hist)
p += (max_phil >= total_phil >= min_phil )

#Proportional constraints
p += (per_scifi >= 0) #can I flag this in interface to <= or >=?
p += (per_read <= 0) 

p.writeLP("ReadingPlan%s.lp" % str(quarter))

start = timeit.default_timer()
results = p.solve(GLPK(msg=0))
gnutime=(timeit.default_timer() - start)
print("Total Solver Time:" + str(gnutime))
#error handling -- throw error? or message?
#assert results == pulp.LpStatusOptimal

print("Status:" + str(LpStatus[p.status]))
print("Optimal at:" + str(value(p.objective)))

#print optimal solution
print("Solution:\n")
msg=""
solution_mask =[]
solution_count = 0

for v in p.variables():
    if v.varValue != 0:
        msg = msg + (v.name + "=" + str(v.varValue) + "\n")
        solution_mask.append(True)
        solution_count += v.varValue
    else:
        solution_mask.append(False)
#print(msg)
#print(solution_count)
# Need to add values for proportions, counts, page count, average-average rating, etc#sol_titles = list(compress(data["Title"],solution_mask))
sol_titles = [i for (i, v) in zip(data["Title"], solution_mask) if v]
print('\n'.join(sol_titles))


Total Solver Time:0.042249917984
Status:Optimal
Optimal at:15265.64
Solution:

Stranger in a Strange Land
Earth Afire (The First Formic War, #2)
Complexity: The Emerging Science at the Edge of Order and Chaos
The Coming
The Host (The Host, #1)
The Empire's Corps (The Empire's Corps, #1)
Angles of Attack (Frontlines #3)
Relentless (The Lost Fleet, #5)
The Human Division (Old Man's War, #5)
Operations Research: Applications and Algorithms (with CD-ROM and InfoTrac)
